In [11]:
# -*- coding: utf-8 -*-
"""
Restaurant Reservation Chatbot: GourmetGo Bot
Self-contained rule-based chatbot for Google Colab.
"""

import re
import datetime
import random

bookings = {}
conversation_context = {}

def extract_date(user_input):
    """Extract date from user input."""
    match = re.search(r'(\b(october|oct|november|nov|december|dec)\s+\d{1,2}(st|nd|rd|th)?\b)|(\d{1,2}/\d{1,2}(/\d{4})?)|(tomorrow)', user_input, re.IGNORECASE)
    if match:
        if "tomorrow" in match.group().lower():
            return (datetime.date.today() + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        return match.group()
    return None

def extract_time(user_input):
    """Extract time from user input."""
    match = re.search(r'\b(\d{1,2}(:\d{2})?\s?(am|pm))\b|\b(\d{1,2}:\d{2})\b', user_input, re.IGNORECASE)
    if match:
        return match.group()
    return None

def extract_people_count(user_input):
    """Extract number of people from user input."""
    match = re.search(r'\b(\d+)\s?(people|persons|pax|person|guest|guests)?\b', user_input, re.IGNORECASE)
    if match:
        return int(match.group(1))
    return None

def extract_name(user_input):
    """Extract customer name from user input."""
    match = re.search(r'\b(is|under|for)\s+([A-Z][a-z]+)\b', user_input)
    if match:
        return match.group(2)
    return None

def handle_booking(user_id, user_input):
    """Handle table booking conversation flow."""
    global conversation_context

    if user_id not in conversation_context or 'intent' not in conversation_context[user_id]:
        conversation_context[user_id] = {'intent': 'book_table', 'stage': 'ask_date', 'details': {}}
        return "Of course! To book a table, what date would you like to reserve?"

    context = conversation_context[user_id]
    stage = context['stage']

    if stage == 'ask_date':
        date = extract_date(user_input)
        if date:
            context['details']['date'] = date
            context['stage'] = 'ask_time'
            return f"Great, a booking for {date}. What time would you like to come in?"
        else:
            return "I'm sorry, I didn't catch the date. Could you please specify a date, like 'October 10th' or 'tomorrow'?"

    elif stage == 'ask_time':
        time = extract_time(user_input)
        if time:
            context['details']['time'] = time
            context['stage'] = 'ask_people'
            return "Perfect. How many people will be in your party?"
        else:
            return "I didn't understand the time. Please provide a time, like '7 PM' or '19:30'."

    elif stage == 'ask_people':
        people = extract_people_count(user_input)
        if people:
            context['details']['people'] = people
            context['stage'] = 'ask_name'
            return "Excellent. And under what name should I make the reservation?"
        else:
            return "Sorry, how many guests will there be?"

    elif stage == 'ask_name':
        name = user_input.strip().title()
        context['details']['name'] = name
        context['stage'] = 'confirm'

        booking_id = f"BK{random.randint(100, 999)}"
        context['booking_id'] = booking_id
        bookings[booking_id] = context['details']

        details = context['details']
        return (f"Thank you, {details['name']}! I've made a reservation for you.\n"
                f"  - Date: {details['date']}\n"
                f"  - Time: {details['time']}\n"
                f"  - Guests: {details['people']} people\n"
                f"Your booking ID is {booking_id}. We look forward to seeing you!")

    return "I'm a bit confused. Let's start over with the booking."

def get_chatbot_response(user_id, user_input):
    """Generate chatbot response based on user input."""
    global conversation_context
    user_input = user_input.lower()

    if user_id in conversation_context and conversation_context[user_id]['intent'] == 'book_table':
        if "cancel" in user_input or "stop" in user_input:
            del conversation_context[user_id]
            return "Okay, I've cancelled the current booking process."
        if conversation_context[user_id].get('stage') != 'confirm':
            return handle_booking(user_id, user_input)
        else:
            del conversation_context[user_id]

    if re.search(r'\b(hello|hi|hey|greetings)\b', user_input):
        return "Hello! Welcome to GourmetGo Bot. You can book, check, or cancel a reservation."

    if re.search(r'\b(book|reservation|reserve|table)\b', user_input):
        return handle_booking(user_id, user_input)

    if re.search(r'\b(check|status|my booking)\b', user_input):
        match = re.search(r'BK\d{3}', user_input.upper())
        if match:
            booking_id = match.group()
            if booking_id in bookings:
                details = bookings[booking_id]
                return (f"Found your booking! Here are the details for {booking_id}:\n"
                        f"  - Name: {details['name']}\n"
                        f"  - Date: {details['date']}\n"
                        f"  - Time: {details['time']}\n"
                        f"  - Guests: {details['people']}")
            else:
                return "I couldn't find a booking with that ID. Please double-check it."
        else:
            return "To check your reservation, please provide your booking ID (e.g., 'check my booking BK123')."

    if re.search(r'\b(cancel|delete|remove)\b', user_input):
        match = re.search(r'BK\d{3}', user_input.upper())
        if match:
            booking_id = match.group()
            if booking_id in bookings:
                del bookings[booking_id]
                return f"Your reservation {booking_id} has been successfully cancelled."
            else:
                return "I couldn't find a booking with that ID to cancel."
        else:
            return "To cancel, please provide your booking ID (e.g., 'cancel my booking BK123')."

    if re.search(r'\b(thanks|thank you)\b', user_input):
        return "You're welcome! Is there anything else I can help you with?"

    if re.search(r'\b(bye|goodbye|see you)\b', user_input):
        return "Goodbye! Have a great day!"

    return "I'm sorry, I don't understand that. I can help with booking, checking, or canceling a reservation."

def main():
    """Run the chatbot interaction loop."""
    print("=" * 50)
    print("    Welcome to GourmetGo Restaurant Reservation Bot!")
    print("=" * 50)
    print("You can start by saying 'hi' or 'I want to book a table'.")
    print("Type 'quit' to exit.")
    print("-" * 50)

    user_id = 'user123'

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Bot: Goodbye!")
            break
        response = get_chatbot_response(user_id, user_input)
        print(f"Bot: {response}")

if __name__ == "__main__":
    main()


    Welcome to GourmetGo Restaurant Reservation Bot!
You can start by saying 'hi' or 'I want to book a table'.
Type 'quit' to exit.
--------------------------------------------------
You: hi
Bot: Hello! Welcome to GourmetGo Bot. You can book, check, or cancel a reservation.
You: I want to book a table.
Bot: Of course! To book a table, what date would you like to reserve?
You: I want to book for tomorrow at around 12:30 pm for 2 in a Thai or a Japanese resturant
Bot: Great, a booking for 2025-10-08. What time would you like to come in?
You: 12:30 pm
Bot: Perfect. How many people will be in your party?
You: 2
Bot: Excellent. And under what name should I make the reservation?
You: Annany
Bot: Thank you, Annany! I've made a reservation for you. Here are the details:
  - Date: 2025-10-08
  - Time: 12:30 pm
  - Guests: 2 people
Your booking ID is BK521. We look forward to seeing you!
You: check status for booking BK521
Bot: Found your booking! Here are the details for BK521:
  - Name: Annany